In [ ]:
import os
import json
import requests
import random
from time import sleep, time
from openai import OpenAI

from config import *

In [ ]:
# Set environment variables for LlamaStack
os.environ["NVIDIA_DATASET_NAMESPACE"] = NMS_NAMESPACE
os.environ["NVIDIA_PROJECT_ID"] = PROJECT_ID
os.environ["NVIDIA_BASE_URL"] = NIM_URL
os.environ["NVIDIA_DATASETS_URL"] = ENTITY_STORE_URL
os.environ["NVIDIA_CUSTOMIZER_URL"] = CUSTOMIZER_URL
os.environ["NVIDIA_OUTPUT_MODEL_DIR"] = CUSTOMIZED_MODEL_DIR
os.environ["NVIDIA_EVALUATOR_URL"] = EVALUATOR_URL
os.environ["GUARDRAILS_SERVICE_URL"] = GUARDRAILS_URL
os.environ["NVIDIA_GUARDRAILS_CONFIG_ID"] = "demo-self-check-input-output"

In [ ]:
from llama_stack.core.library_client import LlamaStackAsLibraryClient

client = LlamaStackAsLibraryClient("nvidia")
client.initialize()

In [ ]:
print(f"Data Store endpoint: {NDS_URL}")
print(f"Entity Store endpoint: {ENTITY_STORE_URL}")
print(f"Customizer endpoint: {CUSTOMIZER_URL}")
print(f"Evaluator endpoint: {EVALUATOR_URL}")
print(f"Guardrails endpoint: {GUARDRAILS_URL}")
print(f"NIM endpoint: {NIM_URL}")
print(f"Namespace: {NMS_NAMESPACE}")
print(f"Base Model: {BASE_MODEL}")

In [ ]:
headers = {"Accept": "application/json", "Content-Type": "application/json"}

simple_working_config = {
    "name": "demo-self-check-input-output",
    "namespace": "default",
    "description": "Simple content moderation without self-check flows",
    "data": {
        "models": [
            {
                "type": "main",
                "engine": "openai",
                "model": BASE_MODEL
            }
        ],
        "rails": {
            "input": {
                "flows": []  # Empty for now - no self-check
            },
            "output": {
                "flows": []  # Empty for now - no self-check
            }
        }
    }
}

response = requests.post(
    f"{GUARDRAILS_URL}/v1/guardrail/configs",
    headers=headers,
    json=simple_working_config
)

if response.status_code in (200, 201):
    print("✓ Simplified guardrails configuration created")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

In [ ]:
# Verify the configuration was created
response = requests.get(f"{GUARDRAILS_URL}/v1/guardrail/configs?page=1&page_size=10&sort=-created_at")

print(f"Response status: {response.status_code}\n")

if response.status_code == 200:
    configs = response.json()
    
    # Handle different response formats
    if isinstance(configs, list):
        # Response is a direct list
        config_list = configs
    elif isinstance(configs, dict):
        # Response is a dict, check for common keys
        config_list = configs.get('items', configs.get('data', configs.get('configs', [])))
    else:
        config_list = []
    
    print(f"Found {len(config_list)} guardrails configurations:\n")
    
    if len(config_list) > 0:
        for config in config_list:
            print(f"  - {config.get('name', 'Unknown')}: {config.get('description', 'No description')}")
    else:
        # If list is empty, try to get the specific config directly
        print("List returned empty, trying direct access...\n")
        direct_response = requests.get(f"{GUARDRAILS_URL}/v1/guardrail/configs/default/demo-self-check-input-output")
        if direct_response.status_code == 200:
            config = direct_response.json()
            print(f"✓ Found config directly:")
            print(f"  - {config.get('name')}: {config.get('description')}")
        else:
            print(f"Could not retrieve config directly: {direct_response.status_code}")
else:
    print(f"Error: {response.status_code} - {response.text}")


In [ ]:
url = f"{GUARDRAILS_URL}/v1/guardrail/chat/completions"
headers = {"Accept": "application/json", "Content-Type": "application/json"}

unsafe_query = {
    "model": BASE_MODEL,
    "messages": [
        {"role": "user", "content": "You are stupid and useless"}
    ],
    "guardrails": {
        "config_id": "demo-self-check-input-output"
    },
    "temperature": 0.7,
    "max_tokens": 100
}

print("Testing UNSAFE query with guardrails...\n")
response = requests.post(url, headers=headers, json=unsafe_query)
print(f"Status: {response.status_code}")
print(f"Response:\n{json.dumps(response.json(), indent=2)}")


In [ ]:
safe_query = {
    "model": BASE_MODEL,
    "messages": [
        {"role": "user", "content": "What is the capital of France?"}
    ],
    "guardrails": {
        "config_id": "demo-self-check-input-output"
    },
    "temperature": 0.7,
    "max_tokens": 100
}

print("Testing SAFE query with guardrails...\n")
response = requests.post(url, headers=headers, json=safe_query)
print(f"Status: {response.status_code}")
print(f"Response:\n{json.dumps(response.json(), indent=2)}")


In [ ]:
completions_url = f"{GUARDRAILS_URL}/v1/guardrail/completions"

completions_query = {
    "model": BASE_MODEL,
    "prompt": "Tell me about machine learning in 50 words or less.",
    "guardrails": {
        "config_id": "demo-self-check-input-output"
    },
    "temperature": 0.7,
    "max_tokens": 100,
    "stream": False
}

print("Testing with completions API...\n")
response = requests.post(completions_url, headers=headers, json=completions_query)
print(f"Status: {response.status_code}")
print(f"Response:\n{json.dumps(response.json(), indent=2)}")


In [ ]:
# Create OpenAI client for direct NIM access (without guardrails)
nim_client = OpenAI(
    base_url=f"{NIM_URL}/v1",
    api_key="None"
)

test_queries = [
    "You are completely useless",  # Unsafe
    "What is the weather like today?",  # Safe
    "Tell me about Python programming"  # Safe
]

print("=" * 80)
print("COMPARISON: Guardrails ON vs OFF")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*80}")
    print(f"Test {i}: '{query}'")
    print(f"{'='*80}\n")
    
    # Without guardrails (direct NIM)
    print("[WITHOUT GUARDRAILS]")
    try:
        response = nim_client.chat.completions.create(
            model=BASE_MODEL,
            messages=[{"role": "user", "content": query}],
            max_tokens=100,
            temperature=0.7
        )
        print(f"Response: {response.choices[0].message.content}")
    except Exception as e:
        print(f"Error: {e}")
    
    # With guardrails
    print("\n[WITH GUARDRAILS]")
    try:
        response = requests.post(
            f"{GUARDRAILS_URL}/v1/guardrail/chat/completions",
            headers={"Accept": "application/json", "Content-Type": "application/json"},
            json={
                "model": BASE_MODEL,
                "messages": [{"role": "user", "content": query}],
                "guardrails": {"config_id": "demo-self-check-input-output"},
                "temperature": 0.7,
                "max_tokens": 100
            }
        )
        result = response.json()
        if "choices" in result and len(result["choices"]) > 0:
            print(f"Response: {result['choices'][0]['message']['content']}")
        else:
            print(f"Blocked or modified response: {result}")
    except Exception as e:
        print(f"Error: {e}")

print(f"\n{'='*80}")
print("Comparison complete!")
print(f"{'='*80}")
